# Landslide Hazard Analysis Using SAR
In this notebook, you will learn how to generate a landslide density map  with RTC-based change detection analysis.  You will create multi-temporal stacks of Sentinel-1 data using [XArray](https://docs.xarray.dev/en/stable/) and compare data from before and after an earthquake that triggered landslides.

In this tutorial, we will focus mapping landslides triggered by an $M_w$ 7.2 [earthquake in Haiti](https://en.wikipedia.org/wiki/2021_Haiti_earthquake) that occurred on August 14, 2021. The earthquake triggered widespread landslides across the southwestern part of the country, with  least 8,444 landslides triggered across a 2,700 $km^2$ (1,000 sq mi) area [(Zhang et al., 2021)](https://www.sciencedirect.com/science/article/abs/pii/S0169555X22003129).  Rapid response to the landslide and inventory of landslides was further hindered by the arrival of Tropical Storm Grace. The presence of consistent cloud coverage makes SAR data an ideal candidate to detect landslides in the aftermath, since SAR can penetrate through cloud cover.

This notebook will show you how to perform the time-series change detection as performed by [Handwerger et al., 2022](https://nhess.copernicus.org/articles/22/753/2022/). This process utilizes OnDemand RTC products from the Alaska Satellite Facility.

In this notebook, we will:
1. Use the HyP3 Python SDK to:
   - Request On Demand RTC products from ASF HyP3
   - Download the RTC products when they are done processing
<br></br>
2. And you will use [Xarray](https://docs.xarray.dev/en/stable/) and various Python utilities to:
   - Download pre-processed data that we will use for the time series
   - Load data with Xarray
   - Group data pre-event and post-event and average both stacks
   - Perform a log difference between pre-event and post-even scenes
   - Plot a histogram of the log difference for a subset of the area
   - Reproduce the main landslide heatmap figure from Handwerger et al., 2022


## 0. Initial Setup

To run this notebook, you'll need to be in the `insar_analysis` conda environment within OpenSARLab.

Alternatively, you can set up your own environment by running these commands in your shell (you'll need to have [conda](https://docs.conda.io/projects/continuumio-conda/en/latest/user-guide/install/index.html) installed):
```shell
curl -OL https://raw.githubusercontent.com/ASFOpenSARlab/opensarlab-envs/main/Environment_Configs/insar_analysis_env.yml
conda env create -f insar_analysis_env.yml
```
Then launch this notebook from the new environment:
```shell
conda activate insar_analysis
jupyter lab igarss_mtedgecumbe_ts_analysis.ipynb
```

Once you have completed the setup for one of these two environments, you are ready to start working with the data.

## 1. Search for SLC images over Haiti in a defined time period

In [ ]:

from datetime import datetime

start_time = datetime.strptime('2020-08-01T23:59', '%Y-%m-%dT%H:%M')
end_time = datetime.strptime('2021-08-16T23:59', '%Y-%m-%dT%H:%M')
event_time = datetime.strptime('2021-08-14T00:00', '%Y-%m-%dT%H:%M')

import asf_search
scenes_to_submit = []
wkt = 'POLYGON((-74.6 18.0,-73.0 18.0,-73.0 18.8,-74.6 18.8, -74.6 18.0))'
results_descending = asf_search.geo_search(platform=[asf_search.PLATFORM.SENTINEL1], intersectsWith=wkt, processingLevel='SLC', start=start_time, end=end_time, relativeOrbit=142, frame=530)
# [scenes_to_submit.append(result.properties['sceneName']) for result in results]
# print(f'There are {len(scenes_to_submit)} scenes in the AOI between {start_time} and {end_time}')

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape

properties = [result.geojson()['properties'] for result in results_descending]
geometries = [shape(result.geojson()['geometry']) for result in results_descending]
gdf = gpd.GeoDataFrame(properties, geometry=geometries,crs='EPSG:4326')
scenes_to_submit = list(gdf['sceneName'])
print(gdf.shape[0])

In [ ]:
gdf.explore(style_kwds=dict(color='black', fill=False))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plot_gdf = gdf[['sceneName', 'startTime']].copy()
plot_gdf['startTime'] = pd.to_datetime(plot_gdf['startTime'])
plot_gdf = plot_gdf.sort_values('startTime').reset_index()

f, ax = plt.subplots(1,1,figsize=(7,7))
ax.scatter(pd.to_datetime(plot_gdf['startTime']), plot_gdf.index, s=5)
ax.set(xlabel='Date', ylabel='Image Number')
ax.axvline(event_time, color='red')

### 2. Acquire Data using HyP3 SDK

### 2.1 Submit jobs from Step 1.

In [ ]:
import hyp3_sdk as sdk

print(f'Submitting {len(scenes_to_submit)} jobs')
hyp3 = sdk.HyP3()
rtc_jobs = sdk.Batch()
for scene in scenes_to_submit:
    rtc_jobs += hyp3.submit_rtc_job(granule=scene, name='IGARSS-HAITI')

In [ ]:
data_dir_path = 'haiti_rtcs'
rtc_jobs = hyp3.watch(rtc_jobs)
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
file_list = succeeded_jobs.download_files(location=data_dir_path)

### 2.2 Download pre-processed data with the HyP3 SDK

In [ ]:
import hyp3_sdk as sdk
from pathlib import Path
import os

hyp3 = sdk.HyP3()

if not os.path.isdir('haiti_rtcs'):
    os.mkdir('haiti_rtcs')
rtc_jobs = hyp3.find_jobs(name='IGARSS-HAITI', user_id ='ffwilliams2')
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
file_list = succeeded_jobs.download_files(location='haiti_rtcs')

files = Path('haiti_rtcs').glob('*')
for file in files:
    sdk.util.extract_zipped_product(file, delete=False)

### 3. Crop all images to same extent

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal


def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

In [ ]:
from pathlib import Path
from typing import List, Union

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """

    
    files_for_mintpy = ['_VV.tif', '_VH.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [ ]:
data_dir = Path('haiti_rtcs')
files = list(data_dir.glob('**/*_VV.tif'))
overlap = get_common_overlap(files)
clip_hyp3_products_to_common_overlap(data_dir, overlap)

### 4. Load the geotiffs into Xarray with datetime stamps for each image

In [ ]:
import xarray as xr
import os
from pathlib import Path

def preprocess(da_orig, file_type: str='vv'):
    '''function that should return an xarray object with time dimension and associated metadata given a path to a single RTC scene, if its dualpol will have multiple bands, currently just as 2 data arrays but could merge.
    goal would be to apply this a list of directories for different RTC products, return cube along time dimension - I think?
    - for concatenating, would need to check footprints and only take products with the same footprint, or subset them all to a common AOI? '''
    da = da_orig.copy()
    da = da.rename({'band_data': file_type}).squeeze()
    fname = os.path.basename(da_orig['band_data'].encoding['source'])
    time = datetime.strptime(fname[7:22], '%Y%m%dT%H%M%S')
    da = da.assign_coords({'time': time})
    da = da.expand_dims('time')
    da = da.drop_duplicates(dim=['x', 'y'])

    return da

fpaths_vv = list(Path('haiti_rtcs').glob('*/*_VV_clipped.tif'))
rtc_vv = xr.open_mfdataset(paths = fpaths_vv, preprocess = preprocess, chunks = 'auto', engine='rasterio', data_vars='minimal', coords='minimal', concat_dim='time', combine='nested', parallel=True)

In [ ]:
rtc_vv['vv']

### 5. Pre-temporal averaging, create time series plot of single pixel (or group of pixels)

### 6. Create a temporal average for both pre and post EQ

In [ ]:
import numpy as np
from datetime import datetime

start_time = datetime.strptime('2020-08-01T23:59', '%Y-%m-%dT%H:%M')
end_time = datetime.strptime('2021-08-16T23:59', '%Y-%m-%dT%H:%M')
event_time = datetime.strptime('2021-08-14T00:00', '%Y-%m-%dT%H:%M')

date_bins = [start_time, event_time, end_time]
date_bin_labels = ['preevent', 'postevent']

rtc_vv = rtc_vv.groupby_bins('time', date_bins, labels=date_bin_labels)
mean_vv = rtc_vv.mean(dim='time')

In [ ]:
mean_vv['vv']

### 7. Perform a log difference of the two resulting images.

In [ ]:
log_diff =  10*np.log10(mean_vv['vv'][0]/mean_vv['vv'][1])
log_diff = log_diff.compute()

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.imshow(log_diff)
plt.colorbar()
plt.show()

### 8. Discuss options for thresholding the difference image

### 9. Use the COP30 DEM to remove change detections in areas with slopes <5 degrees use gdaldem

In [ ]:
from dem_stitcher import stitch_dem


dem_file = data_dir_path / 'DEM.tif'
bounds = [-74.6, 18.0, -73.0, 18.8]
X, p = stitch_dem(bounds,
                  dem_name='glo_30',  # Global Copernicus 30 meter resolution DEM
                  dst_ellipsoidal_height=False,
                  dst_area_or_point='Point')

import rasterio

with rasterio.open(dem_file, 'w', **p) as ds:
   ds.write(X, 1)
   ds.update_tags(AREA_OR_POINT='Point')

process_dem_file = data_dir_path / 'DEM_processed.tif'
gdal.DEMProcessing(destName=str(process_dem_file), srcDS=str(dem_file), processing="slope", format="Gtiff", slopeFormat="degree")

height, width = np.shape(log_diff)
gdal.Warp(str(process_dem_file), str(process_dem_file), dstSRS='EPSG:4326',
                  outputBounds=bounds, width=width, height=height, resampleAlg='nearest', format='GTiff')

dem = gdal.Open(str(process_dem_file), gdal.GA_ReadOnly).ReadAsArray()
dem_mask = dem > 5

log_diff = log_diff.where(dem_mask)

 ### 10. Plot a histogram for the difference image (might have to subset histogram source to area with equal amounts of landslides and non-landslides).

In [ ]:
import matplotlib.pyplot as plt

plt.hist(log_diff)
plt.show()

### 11. Create a figure to interactively view the data and set a threshold using matplotlib

In [ ]:
plt.matshow(log_diff.where(dem_mask))
plt.show()